# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import World Happiness Report Data 2021
city_data = pd.read_csv("../WeatherPy/city_data.csv")
city_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Severo-Kuril'sk,50.6789,156.1250,32.13,98,100,13.31,RU,1619402467
1,Khatanga,71.9667,102.5000,13.82,92,100,14.70,RU,1619402762
2,Lagoa,39.0500,-27.9833,59.16,84,100,10.94,PT,1619402762
3,Sioux Lookout,50.1001,-91.9170,42.80,31,75,3.44,CA,1619402763
4,Punta Arenas,-53.1500,-70.9167,46.40,93,75,2.30,CL,1619402763
...,...,...,...,...,...,...,...,...,...
563,George Town,5.4112,100.3354,86.00,83,20,5.75,MY,1619402716
564,Muros,42.7762,-9.0603,55.00,94,12,4.09,ES,1619402901
565,Alanya,36.5438,31.9998,57.20,77,0,4.61,TR,1619402902
566,Kashan,33.9831,51.4364,72.73,33,100,2.98,IR,1619402902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)

humidity = city_data['Humidity']

In [4]:
# Create a Humidity Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=2)

humidity_heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(humidity_heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Dropna in city_data
city_data_narrowed = city_data.dropna()

# Narrow down the cities to fit weather conditions
city_data_narrowed = city_data_narrowed[(city_data_narrowed['Max Temp'] > 70) & (city_data_narrowed['Max Temp'] < 80)]
city_data_narrowed = city_data_narrowed[city_data_narrowed['Wind Speed'] <= 10]
city_data_narrowed = city_data_narrowed[city_data_narrowed['Cloudiness'] == 0]

city_data_narrowed

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,Boueni,-12.9025,45.0761,77.00,88,0,5.75,YT,1619402782
335,Barkhan,29.8977,69.5256,74.97,26,0,3.09,PK,1619402847
339,Riyadh,24.6877,46.7219,78.80,13,0,4.61,SA,1619402848
381,Suzhou,31.3114,120.6181,71.60,43,0,8.95,CN,1619402820
396,Algiers,36.7525,3.0420,77.00,29,0,4.61,DZ,1619402862


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = city_data_narrowed

# Add "Hotel Name" column
hotel_df['Hotel Name'] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
90,Boueni,-12.9025,45.0761,77.00,88,0,5.75,YT,1619402782,
335,Barkhan,29.8977,69.5256,74.97,26,0,3.09,PK,1619402847,
339,Riyadh,24.6877,46.7219,78.80,13,0,4.61,SA,1619402848,
381,Suzhou,31.3114,120.6181,71.60,43,0,8.95,CN,1619402820,
396,Algiers,36.7525,3.0420,77.00,29,0,4.61,DZ,1619402862,


In [7]:
# Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [8]:
# Hit the Google Places API for each city's coordinates

# Import time function 
import time

# Print header
print('Beginning Data Retrieval')
print('------------------------------------------------------------')

# Loop through the hotel_df Dataframe with the iterrows function
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    params['location'] = f'{lat},{lng}'
    
    print('Processing Record for {} (Index {}):'.format(city_name, index))
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print('The closest hotel in {} is {}'.format(city_name, results[0]['name']))
                
    except (KeyError, IndexError):
        print('City not found. Skipping...')
        print("------------------------------------------------------------")
              
        # Limit waiting time
        time.sleep(1)
        
        pass
        
    print("------------------------------------------------------------")
    
# Print footer
print('------------------------------------------------------------')
print('Data Retrieval Complete ')
print('------------------------------------------------------------')

Beginning Data Retrieval
------------------------------------------------------------
Processing Record for Boueni (Index 90):
The closest hotel in Boueni is LE CHISSIOUA
------------------------------------------------------------
Processing Record for Barkhan (Index 335):
The closest hotel in Barkhan is National Bank of Pakistan (NBP)
------------------------------------------------------------
Processing Record for Riyadh (Index 339):
The closest hotel in Riyadh is Four Seasons Hotel Riyadh At Kingdom Center
------------------------------------------------------------
Processing Record for Suzhou (Index 381):
The closest hotel in Suzhou is Suzhou Marriott Hotel
------------------------------------------------------------
Processing Record for Algiers (Index 396):
The closest hotel in Algiers is Sofitel Algiers Hamma Garden
------------------------------------------------------------
------------------------------------------------------------
Data Retrieval Complete 
---------------

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))